# P2 MAAAI 

PREPROCESADO DE DATOS:

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np

In [3]:


# Cargar CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train.astype('float32') / 255.0  # Normalizar imágenes a [0,1]
x_test = x_test.astype('float32') / 255.0

# Dividir en datos etiquetados y no etiquetados (80% sin etiquetas)
np.random.seed(42)
unlabeled_indices = np.random.choice(len(x_train), size=int(0.8 * len(x_train)), replace=False)
labeled_indices = np.setdiff1d(np.arange(len(x_train)), unlabeled_indices)

x_train_labeled = x_train[labeled_indices]
y_train_labeled = y_train[labeled_indices]
x_train_unlabeled = x_train[unlabeled_indices]

print(f"Etiquetadas: {x_train_labeled.shape}, No etiquetadas: {x_train_unlabeled.shape}")

Etiquetadas: (10000, 32, 32, 3), No etiquetadas: (40000, 32, 32, 3)


EJERCICIO 3 AUTOENCODER:

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, Input
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
# ----- Paso 1: Entrenar el Autoencoder -----

def crear_encoder():
    input_img = Input(shape=(32, 32, 3))
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(input_img)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.00005))(x)
    x = layers.BatchNormalization()(x)
    encoded = layers.MaxPooling2D((2, 2))(x)

    encoder = models.Model(input_img, encoded, name="encoder")
    return encoder

def crear_decoder(encoded):
    x = layers.UpSampling2D((2, 2))(encoded)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    return decoded

def crear_autoencoder():
    input_img = Input(shape=(32, 32, 3))
    encoder = crear_encoder()
    encoded = encoder(input_img)
    decoded = crear_decoder(encoded)
    
    autoencoder = models.Model(input_img, decoded, name="autoencoder")
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder


In [6]:
autoencoder, encoder = crear_autoencoder()
# Entrenar autoencoder solo con imágenes no etiquetadas

early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

autoencoder.fit(x_train_unlabeled, x_train_unlabeled, epochs=50, batch_size=128, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 216s 697ms/step - loss: 0.0271 - val_loss: 0.0375
Epoch 2/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 194s 689ms/step - loss: 0.0112 - val_loss: 0.0148
Epoch 3/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 194s 687ms/step - loss: 0.0081 - val_loss: 0.0086
Epoch 4/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 194s 686ms/step - loss: 0.0072 - val_loss: 0.0082
Epoch 5/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 193s 685ms/step - loss: 0.0063 - val_loss: 0.0061
Epoch 6/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 191s 678ms/step - loss: 0.0058 - val_loss: 0.0084
Epoch 7/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 193s 683ms/step - loss: 0.0059 - val_loss: 0.0076
Epoch 8/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 193s 683ms/step - loss: 0.0058 - val_loss: 0.0059
Epoch 9/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 191s 677ms/step - loss: 0.0055 - val_loss: 0.0059
Epoch 10/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 204s 683ms/step - loss: 0.0052 - val_loss: 0.0051
Epoch 11/50
282/282 ━━━━━━━━━━━━━━━━━━━━ 188s 667ms/step - loss: 0.0053 - val_loss: 0.0063
Epoch 12

In [ ]:
# ----- Paso 2: Entrenar el Clasificador -----

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def crear_clasificador(encoder):
    encoder.trainable = False  # Congelamos el encoder preentrenado
    input_img = encoder.input
    x = encoder.output
    x = layers.Flatten()(x)

    # Capa densa 1 con regularización L2 y dropout
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    # Capa densa 2 más pequeña para reducir overfitting
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    # Capa de salida para clasificación
    output = layers.Dense(100, activation='softmax')(x)

    # Modelo final
    classifier = models.Model(input_img, output, name="clasificador")
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # LR ajustado
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return classifier





In [29]:

# Crear el modelo
classifier = crear_clasificador(encoder)

# Callbacks: EarlyStopping y reducción del learning rate si no mejora
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Entrenar solo con datos etiquetados
classifier.fit(
    x_train_labeled, y_train_labeled,
    batch_size=256,
    epochs=50,
    validation_split=0.1,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 84ms/step - accuracy: 0.0101 - loss: 5.8351 - val_accuracy: 0.0300 - val_loss: 5.2280 - learning_rate: 5.0000e-04
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.0249 - loss: 5.4551 - val_accuracy: 0.0570 - val_loss: 5.0825 - learning_rate: 5.0000e-04
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.0336 - loss: 5.2664 - val_accuracy: 0.0810 - val_loss: 4.9384 - learning_rate: 5.0000e-04
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 97ms/step - accuracy: 0.0512 - loss: 5.1111 - val_accuracy: 0.0860 - val_loss: 4.8150 - learning_rate: 5.0000e-04
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.0626 - loss: 4.9734 - val_accuracy: 0.1060 - val_loss: 4.7203 - learning_rate: 5.0000e-04
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.0803 - loss: 4.8420 - val_accuracy: 0.1180 - val_loss: 4.6401 - learning_rate: 5.0000e-04
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.0915 

In [30]:
loss, accuracy = classifier.evaluate(x_test, y_test, verbose=1)
print(f'Pérdida en test: {loss:.4f}')
print(f'Precisión en test: {accuracy:.4%}')

loss, accuracy = classifier.evaluate(x_train_labeled, y_train_labeled, verbose=1)
print(f'Pérdida en train: {loss:.4f}')
print(f'Precisión en train: {accuracy:.4%}')



313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.2489 - loss: 3.5629
Pérdida en test: 3.5587
Precisión en test: 25.0300%
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7629 - loss: 1.7531
Pérdida en train: 1.9289
Precisión en train: 71.0100%
